In [14]:
import pandas as pd

train = pd.read_csv(r"train_final.csv")

print(f'train set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')

train.head()

train set은 24 개의 feature를 가진 2999 개의 데이터 샘플로 이루어져 있습니다.


,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,Male,Loyal Customer,72,Business travel,Business,1784,1,1,4,...,2,2,3,2,5,2,0,0,0,1
1,2,Male,Loyal Customer,35,Business travel,Business,2228,2,5,5,...,2,2,2,2,4,2,0,0,8,0
2,3,Male,disloyal Customer,37,Business travel,Eco,1970,4,3,4,...,1,3,2,5,1,4,0,1,2,1
3,4,Female,Loyal Customer,20,Personal Travel,Eco,2249,5,5,1,...,1,4,1,4,1,5,0,0,0,1
4,5,Male,disloyal Customer,38,Business travel,Eco,1995,2,2,2,...,5,2,5,3,1,2,0,110,102,0


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 2999 non-null   int64 
 1   Gender                             2999 non-null   object
 2   Customer Type                      2999 non-null   object
 3   Age                                2999 non-null   int64 
 4   Type of Travel                     2999 non-null   object
 5   Class                              2999 non-null   object
 6   Flight Distance                    2999 non-null   int64 
 7   Seat comfort                       2999 non-null   int64 
 8   Departure/Arrival time convenient  2999 non-null   int64 
 9   Food and drink                     2999 non-null   int64 
 10  Gate location                      2999 non-null   int64 
 11  Inflight wifi service              2999 non-null   int64 
 12  Inflig

In [16]:
train_x = train.drop(["id","target", 'Departure/Arrival time convenient', 
    'Gate location', 'Food and drink', 'Inflight wifi service','Flight Distance', 
                      'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'Age'],axis=1) 
train_y = train.target 

In [17]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()): #enumerate는 인덱스와 함께 고유값 반환, enumerate(['a', 'b', 'c'])는 (0, 'a'), (1, 'b'), (2, 'c')와 같은 튜플을 반환
                label_map[key] = i+1  
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) 
    return dataframe

# train 데이터 라벨 인코딩
label_map = make_label_map(train_x) # train 사용해 label map 생성
train_x = label_encoder(train_x, label_map) # train 라벨 인코딩

train_x.head()

,Gender,Customer Type,Type of Travel,Class,Seat comfort,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding
0,1,1,1,1,1,5,5,2,2,3,2,5,2,0
1,1,1,1,1,2,1,2,2,2,2,2,4,2,0
2,1,2,1,2,4,4,1,1,3,2,5,1,4,0
3,2,1,2,2,5,1,1,1,4,1,4,1,5,0
4,1,2,1,2,2,2,5,5,2,5,3,1,2,0


In [19]:
'''from sklearn.preprocessing import MinMaxScaler

num_features = ['Age']

scaler = MinMaxScaler()
train_x[num_features] = scaler.fit_transform(train_x[num_features]) 
train_x.info()

SyntaxError: incomplete input (2213456276.py, line 1)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [21]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.2, random_state = 42)

xgb_model = XGBClassifier()

# StratifiedKFold를 사용한 5-Fold 교차 검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(xgb_model, x_train, y_train, cv=cv, scoring='accuracy')

In [22]:


# 교차 검증 정확도 출력
print(f"교차 검증 정확도: {cv_scores}")
print(f"평균 교차 검증 정확도: {cv_scores.mean()}")

# 테스트 데이터에 대한 예측
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost 정확도: {accuracy}")

accuracy1 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
print(f"XGBoost 정확도(불만족을 불만족으로 예측): {accuracy1}")

accuracy2 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
print(f"XGBoost 정확도(만족을 비행기를 만족으로 예측): {accuracy2}")

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("XGBoost 혼동 행렬:")
print(conf_matrix)

# 분류 보고서 출력
class_report = classification_report(y_test, y_pred)
print("XGBoost 분류 보고서:")
print(class_report)


교차 검증 정확도: [0.91875    0.92291667 0.94166667 0.94791667 0.93319415]
평균 교차 검증 정확도: 0.9328888308977035
XGBoost 정확도: 0.9283333333333333
XGBoost 정확도(불만족을 불만족으로 예측): 0.9361702127659575
XGBoost 정확도(만족을 비행기를 만족으로 예측): 0.9213836477987422
XGBoost 혼동 행렬:
[[264  18]
 [ 25 293]]
XGBoost 분류 보고서:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       282
           1       0.94      0.92      0.93       318

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600



In [23]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# LightGBM 모델 정의
lgb_model = LGBMClassifier()

# StratifiedKFold를 사용한 5-Fold 교차 검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(lgb_model, x_train, y_train, cv=cv, scoring='accuracy')

# 교차 검증 정확도 출력
print(f"교차 검증 정확도: {cv_scores}")
print(f"평균 교차 검증 정확도: {cv_scores.mean()}")

# 테스트 데이터에 대한 예측
lgb_model.fit(x_train, y_train)
y_pred = lgb_model.predict(x_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도: {accuracy}")

accuracy1 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
print(f"LightGBM 정확도(불만족을 불만족으로 예측): {accuracy1}")

accuracy2 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
print(f"LightGBM 정확도(만족을 만족으로 예측): {accuracy2}")

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("LightGBM 혼동 행렬:")
print(conf_matrix)

# 분류 보고서 출력
class_report = classification_report(y_test, y_pred)
print("LightGBM 분류 보고서:")
print(class_report)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1059, number of negative: 860
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 1919, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.551850 -> initscore=0.208148
[LightGBM] [Info] Start training from score 0.208148
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1059, number of negative: 860
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000063 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=tru

In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3]
}

xgb_model = XGBClassifier()
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best accuracy: {grid_search.best_score_}")


KeyboardInterrupt: 